Following learning about simulated annealing, I found it to be a very interesting take on the standard gradient descent algorithms, so attempted to implement it. Firstly, I implemented it for basic functions, defined on the real line. However, I begamn thinking how would I go about implementing it for discrete objects. So, having researched a bit about whether it is ever used for this, I discovered it can solve sudokus, thus attempted to do just this. So I have three sections:
1. implementing it for real functions
2. implementing it to solve sudokus
3. my first failed attempt, which I learned a lot from

(As a final note, there are far more efficient ways to solve sudokus!)

In [2]:
import numpy as np
from numpy import asarray
from numpy import exp
from random import choice
from numpy.random import randn
from numpy.random import rand
from numpy.random import seed
import random
import math
import statistics
from copy import deepcopy

## Section 1

In [6]:
def f(x):
	return 0
 
bounds = np.asarray([[-5.0, 5.0]])

In [7]:
best = bounds[:, 0] + np.random.rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])
# evaluate the initial point
best_eval = f(best)

In [8]:
curr, curr_eval = best, best_eval

In [9]:
def f(x):
    return x[0]**2.0


def simulated_annealing(f, bounds, n_iterations, step_size, temp):

    best = bounds[:, 0] + rand(len(bounds)) * (bounds[:, 1] - bounds[:, 0])

    best_eval = f(best)
    
    curr, curr_eval = best, best_eval
    
    for i in range(n_iterations):
        
        candidate = curr + randn(len(bounds)) * step_size

        candidate_eval = f(candidate)

        if candidate_eval < best_eval:
            best, best_eval = candidate, candidate_eval
            
            print('>%d f(%s) = %.5f' % (i, best, best_eval))
        
        diff = candidate_eval - curr_eval
       
        t = temp / float(i + 1)
        
        metropolis = exp(-diff / t)
       
        if diff < 0 or rand() < metropolis:
           
            curr, curr_eval = candidate, candidate_eval
    return [best, best_eval]


seed(1)
bounds = asarray([[-5.0, 5.0]])
n_iterations = 1000
step_size = 0.1
temp = 10
best, score = simulated_annealing(f, bounds, n_iterations, step_size, temp)
print('Done!')
print('f(%s) = %f' % (best, score))


>34 f([-0.78753544]) = 0.62021
>35 f([-0.76914239]) = 0.59158
>37 f([-0.68574854]) = 0.47025
>39 f([-0.64797564]) = 0.41987
>40 f([-0.58914623]) = 0.34709
>41 f([-0.55446029]) = 0.30743
>42 f([-0.41775702]) = 0.17452
>43 f([-0.35038542]) = 0.12277
>50 f([-0.15799045]) = 0.02496
>66 f([-0.11089772]) = 0.01230
>67 f([-0.09238208]) = 0.00853
>72 f([-0.09145261]) = 0.00836
>75 f([-0.05129162]) = 0.00263
>93 f([-0.02854417]) = 0.00081
>144 f([0.00864136]) = 0.00007
>149 f([0.00753953]) = 0.00006
>167 f([-0.00640394]) = 0.00004
>225 f([-0.00044965]) = 0.00000
>503 f([-0.00036261]) = 0.00000
>512 f([0.00013605]) = 0.00000
Done!
f([0.00013605]) = 0.000000


## Section 2

In [5]:
startingSudoku = """
                    012430000
                    304005002
                    000000039
                    001700690
                    980040025
                    063008100
                    270000000
                    600100803
                    000096270
                """

sudoku = np.array([[int(i) for i in line] for line in startingSudoku.split()])

def Print(sudoku):
    print("\n")
    for i in range(len(sudoku)):
        line = ""
        if i == 3 or i == 6:
            print("---------------------")
        for j in range(len(sudoku[i])):
            if j == 3 or j == 6:
                line += "| "
            line += str(sudoku[i,j])+" "
        print(line)

def FixSudokuValues(fixed_sudoku):
    for i in range (0,9):
        for j in range (0,9):
            if fixed_sudoku[i,j] != 0:
                fixed_sudoku[i,j] = 1
    
    return(fixed_sudoku)
    
def CalculateNumberOfErrors(sudoku):
    no_errors = 0 
    for i in range (0,9):
        no_errors += CalculateNumberOfErrorsRowColumn(i ,i ,sudoku)
    return(no_errors)

def CalculateNumberOfErrorsRowColumn(row, column, sudoku):
    no_errors = (9 - len(np.unique(sudoku[:,column]))) + (9 - len(np.unique(sudoku[row,:])))
    return(no_errors)


def CreateList3x3Blocks ():
    final_list = []
    for r in range (0,9):
        temp_list = []
        block1 = [i + 3*((r)%3) for i in range(0,3)]
        block2 = [i + 3*math.trunc((r)/3) for i in range(0,3)]
        for x in block1:
            for y in block2:
                temp_list.append([x,y])
        final_list.append(temp_list)
    return(final_list)

def RandomlyFill3x3Blocks(sudoku, list_of_blocks):
    for block in list_of_blocks:
        for box in block:
            if sudoku[box[0],box[1]] == 0:
                current_block = sudoku[block[0][0]:(block[-1][0]+1),block[0][1]:(block[-1][1]+1)]
                sudoku[box[0],box[1]] = choice([i for i in range(1,10) if i not in current_block])
    return sudoku

def SumOfOneBlock (sudoku, one_block):
    finalSum = 0
    for box in one_block:
        finalSum += sudoku[box[0], box[1]]
    return(finalSum)

def TwoRandomBoxesWithinBlock(fixed_sudoku, block):
    while (1):
        firstBox = random.choice(block)
        secondBox = choice([box for box in block if box is not firstBox ])

        if fixed_sudoku[firstBox[0], firstBox[1]] != 1 and fixed_sudoku[secondBox[0], secondBox[1]] != 1:
            return([firstBox, secondBox])

def FlipBoxes(sudoku, boxesToFlip):
    proposed_sudoku = np.copy(sudoku)
    place_holder = proposed_sudoku[boxesToFlip[0][0], boxesToFlip[0][1]]
    proposed_sudoku[boxesToFlip[0][0], boxesToFlip[0][1]] = proposed_sudoku[boxesToFlip[1][0], boxesToFlip[1][1]]
    proposed_sudoku[boxesToFlip[1][0], boxesToFlip[1][1]] = place_holder
    return (proposed_sudoku)

def ProposedState (sudoku, fixed_sudoku, list_of_blocks):
    random_block = random.choice(list_of_blocks)

    if SumOfOneBlock(fixed_sudoku, random_block) > 6:  
        return(sudoku, 1, 1)
    boxesToFlip = TwoRandomBoxesWithinBlock(fixed_sudoku, random_block)
    proposed_sudoku = FlipBoxes(sudoku,  boxesToFlip)
    return([proposed_sudoku, boxesToFlip])

def ChooseNewState (currentSudoku, fixed_sudoku, list_of_blocks, sigma):
    proposal = ProposedState(currentSudoku, fixed_sudoku, list_of_blocks)
    newSudoku = proposal[0]
    boxesToCheck = proposal[1]
    currentCost = CalculateNumberOfErrorsRowColumn(boxesToCheck[0][0], boxesToCheck[0][1], currentSudoku) + CalculateNumberOfErrorsRowColumn(boxesToCheck[1][0], boxesToCheck[1][1], currentSudoku)
    newCost = CalculateNumberOfErrorsRowColumn(boxesToCheck[0][0], boxesToCheck[0][1], newSudoku) + CalculateNumberOfErrorsRowColumn(boxesToCheck[1][0], boxesToCheck[1][1], newSudoku)
    # currentCost = CalculateNumberOfErrors(currentSudoku)
    # newCost = CalculateNumberOfErrors(newSudoku)
    costDifference = newCost - currentCost
    rho = math.exp(-costDifference/sigma)
    if(np.random.uniform(1,0,1) < rho):
        return([newSudoku, costDifference])
    return([currentSudoku, 0])


def ChooseNumberOfItterations(fixed_sudoku):
    numberOfItterations = 0
    for i in range (0,9):
        for j in range (0,9):
            if fixed_sudoku[i,j] != 0:
                numberOfItterations += 1
    return numberOfItterations

def CalculateInitialSigma (sudoku, fixed_sudoku, list_of_blocks):
    listOfDifferences = []
    current_sudoku = sudoku
    for i in range(1,10):
        current_sudoku = ProposedState(current_sudoku, fixed_sudoku, list_of_blocks)[0]
        listOfDifferences.append(CalculateNumberOfErrors(current_sudoku))
    return (statistics.pstdev(listOfDifferences))


def solveSudoku (sudoku):
    solutionFound = 0
    h=0
    while (solutionFound == 0):
        decreaseFactor = 0.99
        stuckCount = 0
        fixed_sudoku = np.copy(sudoku)
        FixSudokuValues(fixed_sudoku)
        list_of_blocks = CreateList3x3Blocks()
        current_sudoku = RandomlyFill3x3Blocks(sudoku, list_of_blocks)
        sigma = CalculateInitialSigma(sudoku, fixed_sudoku, list_of_blocks)
        score = CalculateNumberOfErrors(current_sudoku)
        itterations = ChooseNumberOfItterations(fixed_sudoku)
        h+=1
        if score <= 0:
            solutionFound = 1
        
        while solutionFound == 0:
            previousScore = score
            for i in range (0, itterations):
                newState = ChooseNewState(current_sudoku, fixed_sudoku, list_of_blocks, sigma)
                current_sudoku = newState[0]
                scoreDiff = newState[1]
                score += scoreDiff
                if score <= 0:
                    solutionFound = 1
                    break

            sigma *= decreaseFactor
            if score <= 0:
                solutionFound = 1
                break
            if score >= previousScore:
                stuckCount += 1
            else:
                stuckCount = 0
            if (stuckCount > 80):
                sigma += 2
            if(CalculateNumberOfErrors(current_sudoku)==0):
                PrintSudoku(current_sudoku)
                break

    return(current_sudoku)

solution = solveSudoku(sudoku)
Print(solution)



7 1 2 | 4 3 9 | 5 8 6 
3 9 4 | 8 6 5 | 7 1 2 
8 5 6 | 2 1 7 | 4 3 9 
---------------------
4 2 1 | 7 5 3 | 6 9 8 
9 8 7 | 6 4 1 | 3 2 5 
5 6 3 | 9 2 8 | 1 4 7 
---------------------
2 7 5 | 3 8 4 | 9 6 1 
6 4 9 | 1 7 2 | 8 5 3 
1 3 8 | 5 9 6 | 2 7 4 


## Section 3

In [10]:
class Sudoku:

    def __init__(self, r1, r2, r3, r4, r5, r6, r7, r8, r9): 
        self.r1 = r1
        self.r2 = r2
        self.r3 = r3
        self.r4 = r4
        self.r5 = r5
        self.r6 = r6
        self.r7 = r7
        self.r8 = r8
        self.r9 = r9
        self.rows = [r1, r2, r3, r4, r5, r6, r7, r8, r9]
        self.c1 = np.array([r1[0], r2[0], r3[0], r4[0], r5[0], r6[0], r7[0], r8[0], r9[0]])
        self.c2 = np.array([r1[1], r2[1], r3[1], r4[1], r5[1], r6[1], r7[1], r8[1], r9[1]])
        self.c3 = np.array([r1[2], r2[2], r3[2], r4[2], r5[2], r6[2], r7[2], r8[2], r9[2]])
        self.c4 = np.array([r1[3], r2[3], r3[3], r4[3], r5[3], r6[3], r7[3], r8[3], r9[3]])
        self.c5 = np.array([r1[4], r2[4], r3[4], r4[4], r5[4], r6[4], r7[4], r8[4], r9[4]])
        self.c6 = np.array([r1[5], r2[5], r3[5], r4[5], r5[5], r6[5], r7[5], r8[5], r9[5]])
        self.c7 = np.array([r1[6], r2[6], r3[6], r4[6], r5[6], r6[6], r7[6], r8[6], r9[6]])
        self.c8 = np.array([r1[7], r2[7], r3[7], r4[7], r5[7], r6[7], r7[7], r8[7], r9[7]])
        self.c9 = np.array([r1[8], r2[8], r3[8], r4[8], r5[8], r6[8], r7[8], r8[8], r9[8]])
        self.b1 = np.array([r1[0], r1[1], r1[2], r2[0], r2[1], r2[2], r3[0], r3[1], r3[2]])
        self.b2 = np.array([r4[0], r4[1], r4[2], r5[0], r5[1], r5[2], r6[0], r6[1], r6[2]])
        self.b3 = np.array([r7[0], r7[1], r7[2], r8[0], r8[1], r8[2], r9[0], r9[1], r9[2]])
        self.b4 = np.array([r1[3], r1[4], r1[5], r2[3], r2[4], r2[5], r3[3], r3[4], r3[5]])
        self.b5 = np.array([r4[3], r4[4], r4[5], r5[3], r5[4], r5[5], r6[3], r6[4], r6[5]])
        self.b6 = np.array([r7[3], r7[4], r7[5], r8[3], r8[4], r8[5], r9[3], r9[4], r9[5]])
        self.b7 = np.array([r1[6], r1[7], r1[8], r2[6], r2[7], r2[8], r3[6], r3[7], r3[8]])
        self.b8 = np.array([r4[6], r4[7], r4[8], r5[6], r5[7], r5[8], r6[6], r6[7], r6[8]])
        self.b9 = np.array([r7[6], r7[7], r7[8], r8[6], r8[7], r8[8], r9[6], r9[7], r9[8]])
        self.array = np.concatenate([self.r1, self.r2, self.r3, self.r4, self.r5, self.r6, self.r7, self.r8, self.r9])
        self.looseboxes = [self.fixed_boxes(self.r1), self.fixed_boxes(self.r2), self.fixed_boxes(self.r3), self.fixed_boxes(self.r4), self.fixed_boxes(self.r5), self.fixed_boxes(self.r6), self.fixed_boxes(self.r7), self.fixed_boxes(self.r8), self.fixed_boxes(self.r9)]
        self.columns = [self.c1, self.c2, self.c3, self.c4, self.c5, self.c6, self.c7, self.c8, self.c9]
        self.boxes = [self.b1, self.b2, self.b3, self.b4, self.b5, self.b6, self.b7, self.b8, self.b9]
        

    @classmethod
    def from_input(cls):
        return cls(
            [int(i) for i in input("Row 1:").split()],
            [int(i) for i in input("Row 2:").split()], 
            [int(i) for i in input("Row 3:").split()],
            [int(i) for i in input("Row 4:").split()],
            [int(i) for i in input("Row 5:").split()],
            [int(i) for i in input("Row 6:").split()],
            [int(i) for i in input("Row 7:").split()],
            [int(i) for i in input("Row 8:").split()],
            [int(i) for i in input("Row 9:").split()],
        )
          
    def fixed_boxes(cls, array):
        fixed_boxes=[]
        for i in range(len(array)):
            if array[i]==0:
                fixed_boxes.append(i)
        return fixed_boxes